In [1]:
import tkinter as tk
from tkinter import filedialog, Label, Canvas
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import pyttsx3

# Load the pre-trained model for classification
model_classification = tf.keras.models.load_model('model.h5')

# Load a simple segmentation model (example)
# This is just a placeholder, replace with an actual segmentation model
def simple_segmentation(image):
    # Example: Thresholding for demonstration
    threshold = 0.5
    seg_image = np.mean(image, axis=-1, keepdims=True) > threshold
    seg_image = np.squeeze(seg_image, axis=-1).astype(np.uint8) * 255
    return seg_image

# CIFAR-10 classes
classes = [
    "Airplane", "Automobile", "Bird", "Cat", "Deer",
    "Dog", "Frog", "Horse", "Ship", "Truck"
]

# Function to load and preprocess the image
def load_image(file_path):
    img = Image.open(file_path)
    img = img.resize((32, 32))  # CIFAR-10 images are 32x32
    img = np.array(img) / 255.0  # Normalize to [0, 1]
    return img

# Function to classify the image and update result_label
def classify_image(file_path):
    img = load_image(file_path)
    
    # Clear previous prediction text
    result_label.config(text="")
    
    # Display the original image
    orig_img = Image.open(file_path)
    orig_img = orig_img.resize((200, 200))  # Resize for display
    orig_img_tk = ImageTk.PhotoImage(orig_img)
    panel.config(image=orig_img_tk)
    panel.image = orig_img_tk
    panel_title.pack()  # Show original image title

    # Perform segmentation (replace with actual segmentation model)
    seg_image = simple_segmentation(img)
    seg_img = Image.fromarray(seg_image)
    seg_img = seg_img.resize((200, 200))  # Resize for display
    seg_img_tk = ImageTk.PhotoImage(seg_img)
    seg_panel.config(image=seg_img_tk)
    seg_panel.image = seg_img_tk
    seg_panel_title.pack()  # Show segmented image title

    # Pause for a moment to ensure the images and labels are displayed
    root.update_idletasks()
    root.after(1000)  # Pause for 1 second before making prediction

    # Classification
    prediction = model_classification.predict(np.expand_dims(img, axis=0))
    confidence = np.max(prediction)
    class_index = np.argmax(prediction)
    if confidence < 0.7:  # Adjust the threshold as needed
        speak("Image cannot be classified")
        result_label.config(text="Prediction: Image cannot be classified")
    else:
        predicted_class = classes[class_index]
        speak(f"I predict this is a {predicted_class} with {confidence:.2f} confidence")
        result_label.config(text=f"Prediction: {predicted_class} (Confidence: {confidence:.2f})")
    result_label.place(relx=0.5, rely=0.2, anchor=tk.CENTER)  # Show result label after prediction

# Function for text-to-speech output
def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)  # Speed percent (can go over 100)
    engine.say(text)
    engine.runAndWait()

# Function to browse and select an image file
def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        speak("Please wait while I analyze the image.")
        classify_image(file_path)

# Function to welcome the user
def welcome_message():
    speak("Welcome to the image classifier and image segmentation application. Please upload an image.")

# Create the main window
root = tk.Tk()
root.title("CIFAR-10 Image Classifier with Segmentation")

# Get screen width and height
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Set window size to full screen
root.geometry(f"{screen_width}x{screen_height}+0+0")

# Load the background image (assuming it exists and is properly managed)
background_image = Image.open("backg.jfif")
background_image = background_image.resize((screen_width, screen_height), Image.LANCZOS)
background_photo = ImageTk.PhotoImage(background_image)

# Create a canvas and set the background image
canvas = Canvas(root, width=screen_width, height=screen_height)
canvas.pack(fill="both", expand=True)
canvas.create_image(0, 0, anchor="nw", image=background_photo)

# Create and place the image panel for original image
panel_frame = tk.Frame(root, bg='lightblue')
panel_frame.place(relx=0.25, rely=0.5, anchor=tk.CENTER)

panel_title = Label(panel_frame, text="Original Image", font=("Times New Roman", 16), bg='lightblue')
panel_title.pack(pady=10)
panel_title.pack_forget()  # Hide initially

panel = Label(panel_frame)
panel.pack()

# Create and place the image panel for segmentation result
seg_panel_frame = tk.Frame(root, bg='lightblue')
seg_panel_frame.place(relx=0.75, rely=0.5, anchor=tk.CENTER)

seg_panel_title = Label(seg_panel_frame, text="Segmented Image", font=("Times New Roman", 16), bg='lightblue')
seg_panel_title.pack(pady=10)
seg_panel_title.pack_forget()  # Hide initially

seg_panel = Label(seg_panel_frame)
seg_panel.pack()

# Create the result label but do not place it yet
result_label = Label(root, text="Prediction: ", font=("Times New Roman", 16), bg='lightblue')

# Create and place the button to browse files
browse_button = tk.Button(root, text="Browse Image", command=browse_file, bg='lightblue', font=("Times New Roman", 16))
browse_button.place(relx=0.5, rely=0.85, anchor=tk.CENTER)

# Start the GUI event loop
root.after(2500, welcome_message)  # Schedule welcome message after 1 second
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 847ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
